In [2]:
from revenues import *
import copy
from multiprocessing import Pool, cpu_count
import sys
from sympy.core.numbers import Float

aStd, aFarming = var("aStd, aFarming")
# these symbols are declared here in order to differentiate standard
# operation from mining, because they have different ages.
eCostStd, eCostFarming, costStd, costFarming, RwStd = var("eCostStd, eCostFarming, costStd, costFarming, RwStd")
subs = [
    (eCostStd, eCost.subs(u, 1).subs(age, aStd)),
    (eCostFarming, eCost.subs(u, 0).subs(age, aFarming)),
    (costStd, cost.subs(age, aStd)),
    (costFarming, cost.subs(age, aFarming)),
    (RwStd, R_w.subs(age, aStd).subs(u, 1))
]  

In [3]:
# Busy POET
params = valSubs

## 
# Three options:
# - useful work
# - standard mining
# - farming

# Useful work is in revenue.py
bestAgeUseful, revUseful = useful_work(params)

# Standard mining
# note we introduce overheadPoET to the cost of standard mining
perCPUCostStandard = (eCostStd + overheadStd + costStd)
perCPURev = (.9*RwStd + R_b - perCPUCostStandard)
cpuCountStandard = (total_cap / perCPUCostStandard).subs(params)
revStandard = (perCPURev * cpuCountStandard)

# Farming: (Mining in standard form doesn't make sense)
perCPUCostFarm = (eCostFarming + overheadFarm + costFarming)
perCPURevFarm = (R_b - perCPUCostFarm)
cpuCountFarm = (total_cap / perCPUCostFarm).subs(params)
revFarm = (perCPURevFarm * cpuCountFarm)



# equilibrium:
numStdMiner = var("numStdMiner")
numFarmer = var("numFarmer")
busyPoET_R_b = annualCryptoRev / ( numStdMiner * cpuCountStandard + numFarmer * cpuCountFarm )
busyPoET_R_b = busyPoET_R_b.subs(subs).subs(params)

In [8]:
aSingleMiner = var("aSingleMiner")

singleMinerUtility = revStandard\
    .subs(subs)\
    .subs(aStd, aSingleMiner)\
    .subs(R_b, busyPoET_R_b)\
    .subs(params)
    
    
condAgeStdOptimal = diff(singleMinerUtility, aSingleMiner).subs(aSingleMiner, aStd)

singleFarmerUtility = revFarm\
    .subs(subs)\
    .subs(aFarming, aSingleMiner)\
    .subs(R_b, busyPoET_R_b)\
    .subs(params)
    
condAgeFarmingOptimal = diff(singleFarmerUtility, aSingleMiner).subs(aSingleMiner, aFarming)

condRevStdEqUseful = revStandard.subs(R_b,busyPoET_R_b).subs(subs).subs(params) - revUseful
condRevFarmingEqUseful = revFarm.subs(R_b,busyPoET_R_b).subs(subs).subs(params) - revUseful

pprint (condAgeStdOptimal.atoms(Symbol))
pprint (condAgeFarmingOptimal.atoms(Symbol))
pprint (condRevStdEqUseful.atoms(Symbol))
pprint (condRevFarmingEqUseful.atoms(Symbol))

print solve([condAgeStdOptimal, condAgeFarmingOptimal, condRevStdEqUseful, condRevFarmingEqUseful], 
            [aFarming, aStd, numFarmer, numStdMiner])

set([aFarming, aStd, numFarmer, numStdMiner])
set([aFarming, aStd, numFarmer, numStdMiner])
set([aFarming, aStd, numFarmer, numStdMiner])
set([aFarming, aStd, numFarmer, numStdMiner])


KeyboardInterrupt: 

In [ ]:



# fully expand now
    busyPoET_R_b = busyPoET_R_b.subs(subs)
    equiStdOpRatio = equiStdOpRatio.subs(subs)

    aF = MAX_STALENESS / 2
    aS = MAX_STALENESS / 2
    turn = 'S'
    ageStdDone = False
    ageFarmingDone = False

    while True:
        # print 'In turn %s aF=%.2f and aS=%.2f' % (turn, aF, aS)
        if turn == 'S' and not ageStdDone:
            tmp = aS
            # one guy works at aSS, others all work at aOthers
            # R_b and equiStdOpRatio is determined by aOthers
            f = lambda aOthers: aOthers - minimize_scalar(
                lambdify(aStd,
                        -(revStandard
                        .subs(R_b, busyPoET_R_b.subs(aStd, aOthers))
                        .subs(stdOpRatio, equiStdOpRatio.subs(aStd, aOthers))
                        .subs(subs).subs(params)
                        .subs(aFarming, aF))),
                bounds=(0, MAX_STALENESS),
                method='bounded').x

            aS = fsolve(f, MAX_STALENESS / 2)
            if abs(tmp - aS) < 0.01:
                ageStdDone = True
            turn = 'F'
        elif turn == 'F' and not ageFarmingDone:
            tmp = aF
            # one guy works at aFF, others all work at aOthers
            # R_b and equiStdOpRatio is determined by aOthers
            f = lambda aOthers: aOthers - minimize_scalar(
                    lambdify(aFarming,
                            -(revFarm.subs(R_b, busyPoET_R_b.subs(aFarming, aOthers))
                                .subs(stdOpRatio, equiStdOpRatio.subs(aFarming, aOthers))
                                .subs(subs).subs(params)
                                .subs(aStd, aS))),
                    bounds=(0, MAX_STALENESS),
                    method='bounded').x
            aF = fsolve(f, MAX_STALENESS / 2)
            if abs(tmp - aF) < 0.01:
                ageFarmingDone = True
            turn = 'S'
        else:
            break


    equiStdOpRatio_v = equiStdOpRatio.subs(subs).subs(params).subs(aStd, aS).subs(aFarming, aF)
    print 'Find equilibrium %.2f' % (equiStdOpRatio_v)

    if equiStdOpRatio_v <= 0:
        equiStdOpRatio_v = 0
        # if no standard mining at all
        f = lambda aOthers: aOthers - minimize_scalar(
                lambdify(aFarming,
                        -(revFarm
                        .subs(R_b, busyPoET_R_b.subs(aFarming, aOthers))
                        .subs(stdOpRatio, equiStdOpRatio_v)
                        .subs(subs)
                        .subs(params))),
                bounds=(0, MAX_STALENESS),
                method='bounded').x
        aF = fsolve(f, MAX_STALENESS / 2)[0]
        aS = -1
    elif equiStdOpRatio_v >= 1:
        equiStdOpRatio_v = 1
        f = lambda aOthers: aOthers - minimize_scalar(
                lambdify(aStd,
                        -(revStandard
                        .subs(R_b, busyPoET_R_b.subs(aStd, aOthers))
                        .subs(stdOpRatio, equiStdOpRatio_v)
                        .subs(subs)
                        .subs(params))),
                bounds=(0, MAX_STALENESS),
                method='bounded').x
        aS = fsolve(f, MAX_STALENESS / 2)[0]
        aF = -1

    print "Result: annualCryptoRev %.2f aStd %.2f ; aFarming %.2f equilibrium is %.2f" % (
        params["annualCryptoRev"], aS, aF, equiStdOpRatio_v)


    v_busypoet_Rb = busyPoET_R_b\
            .subs(stdOpRatio, equiStdOpRatio_v)\
            .subs(aFarming, aF)\
            .subs(aStd, aS)\
            .subs(params)

# if one guy is farming using an older chip
    # revStandard_v = revStandard\
    #         .subs(R_b, v_busypoet_Rb)\
    #         .subs(stdOpRatio, equiStdOpRatio_v)\
    #         .subs(subs)\
    #         .subs(params)\
    #         .subs(aStd, aS)\
    #         .subs(aFarming, aF)

    # revFarm_v = revFarm\
    #         .subs(R_b, v_busypoet_Rb)\
    #         .subs(stdOpRatio, equiStdOpRatio_v)\
    #         .subs(subs)\
    #         .subs(params)\
    #         .subs(aStd, aS)\
    #         .subs(aFarming, aF)

    # print("Busy PoET revStandard@%.2f: %s%.2f%s" % (aS, colors.BOLD, revStandard_v, colors.ENDC))
    # print("Busy PoET revFarm@%.2f: %s%.2f%s" % (aF, colors.BOLD, revFarm_v, colors.ENDC))

    usefulPrice = (total_cap / (equiStdOpRatio_v*cpuCountStandard*perfSlowdownExpression)
        ).subs(subs).subs(valSubs).subs(age, aS).subs(aStd, aS) / usefulPriceBaseline

    if equiStdOpRatio_v == 0:
# if no one is doing useful work
        usefulPrice = -1
        revFarm_10_v = revFarm.subs(R_b, v_busypoet_Rb).subs(stdOpRatio, equiStdOpRatio_v).subs(subs).subs(params).subs(aFarming, MAX_STALENESS)
        print("Busy PoET revFarm@%.2f: %s%.2f%s" % (MAX_STALENESS, colors.BOLD, revFarm_10_v, colors.ENDC))

    print 'Usefulprice is', usefulPrice
    return (params["annualCryptoRev"], params['operatorCount'], aS, aF, equiStdOpRatio_v, usefulPrice)

In [ ]:
def test(params):
    return params['annualCryptoRev']

def varyAnnaulCryptoRev(func, minRev, maxRev, n):
    params = []
    for an in np.linspace(minRev, maxRev, n):
        tmp = copy.copy(valSubs)
        tmp["annualCryptoRev"] = an
        params.append(tmp)

    pool = Pool(processes=cpu_count())
    results = np.array(pool.map(func, params))
    np.savetxt('../oakland/figs/%s-rev.txt' % func.__name__,
            results, fmt='%.4f',
            header='annualCryptoRev, operatorCount, aUseful, aStd, equil, usefulPrice')

def varyOperatorCount(func):
    params = []
    for an in np.linspace(10, 5000, 50, dtype=int):
        tmp = copy.copy(valSubs)
        tmp["operatorCount"] = float(an)
        params.append(tmp)

    pool = Pool(processes=cpu_count())
    results = np.array(pool.map(func, params))
    np.savetxt('../oakland/figs/%s-opcount.txt' % func.__name__,
            results, fmt='%.4f',
            header='annualCryptoRev, operatorCount, aUseful, aStd, equil, usefulPrice')

# if __name__ == '__main__':
#     # varyAnnaulCryptoRev(lazy_poet, 100000, 3000000, 50)
#     varyAnnaulCryptoRev(busy_poet, 100000, 3000000, 10)

In [8]:
def busy_poet_rb(aS, aF, nS, nF):
    return busyPoET_R_b.subs(subs).subs(params).subs([
            (aStd, aS),
            (aFarming, aF),
            (numStdMiner, nS),
            (numFarmer, nF)])

def condRevEqual1(ageStd, ageFarm, numStd, numFarm):
    Rb = busy_poet_rb(ageStd, ageFarm, numStd, numFarm);
    return (revStandard - revUseful).subs(R_b, Rb).subs(subs).subs(params).subs([
            (aStd, ageStd),
            (aFarming, ageFarm),
            (numStdMiner, numStd),
            (numFarmer, numFarm)])
    
def condRevEqual2(ageStd, ageFarm, numStd, numFarm):
    Rb = busy_poet_rb(ageStd, ageFarm, numStd, numFarm);
    return (revFarm - revUseful).subs(R_b, Rb).subs(subs).subs(params).subs([
            (aStd, ageStd),
            (aFarming, ageFarm),
            (numStdMiner, numStd),
            (numFarmer, numFarm)])

def condOptimalAgeStd(ageStd, ageFarm, numStd, numFarm):
    Rb = busy_poet_rb(ageStd,ageFarm,numStd,numFarm)
#     print 'in condOptimalAgeStd: %.2f, %.2f, %.2f, %.2f, %.2f' % (Rb, ageStd,ageFarm,numStd,numFarm)
    r = abs(ageStd - minimize_scalar(
        lambda ageStdPrime: -1*revStandard.subs(R_b, Rb).subs(subs).subs(params).subs(aStd, ageStdPrime),
        bounds=(0, MAX_STALENESS),
        method='bounded').x)
#     print 'in condOptimalAgeStd: -> %.2f' % (r)
    return r


def condOptimalAgeFarm(ageStd, ageFarm, numStd, numFarm):
    Rb = busy_poet_rb(ageStd,ageFarm,numStd,numFarm)
#     print 'in condOptimalAgeFarm: %.2f, %.2f, %.2f, %.2f, %.2f' % (Rb, ageStd,ageFarm,numStd,numFarm)
    r = abs(ageFarm - minimize_scalar(
        lambda ageFarmPrime: -1*revFarm.subs(R_b, Rb).subs(subs).subs(params).subs(aFarming, ageFarmPrime),
        bounds=(0, MAX_STALENESS),
        method='bounded').x)

#     print 'in condOptimalAgeFarm: -> %.2f' % (r)
    return r

def conditions(x):
    ageStd, ageFarm, numStd, numFarm = x
    ageStd = min(10, max(0, ageStd))
    r = (condRevEqual1(ageStd, ageFarm, numStd, numFarm),
           condRevEqual2(ageStd, ageFarm, numStd, numFarm),
           condOptimalAgeStd(ageStd, ageFarm, numStd, numFarm),
           condOptimalAgeFarm(ageStd, ageFarm, numStd, numFarm))
    print '%.2f '*4 % tuple(x)
    return r

In [9]:
fsolve(conditions, [5,5,10000,10000])

5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.41 96.96 -44512684.34 13578043.69 
4.76 164.57 -22797855.75 6611016.08 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.00 5.00 10000.00 10000.00 
5.41 96.96 -44512684.34 13578043.69 
4.76 164.57 -22797855.75 6611016.08 
4.61 208.11 -11896120.45 3123184.47 
4.51 233.22 -6376223.00 1364230.11 
4.46 242.97 -3477508.39 454422.77 
4.52 225.63 -1709404.61 -50187.79 
5.14 53.95 231451.72 -279472.47 
4.93 -19.48 -100725.86 154736.23 


/Users/fanz/Library/Python/2.7/lib/python/site-packages/scipy/optimize/minpack.py:236: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


array([  5.00000000e+00,   5.00000000e+00,   1.00000000e+04,
         1.00000000e+04])